In [1]:
import torch
from torchvision import transforms, utils
import pandas as pd
import numpy as np
import csv

import cxr_dataset as CXR
import eval_model as E
import metrics_model
import model as M
import argparse


from tqdm import tqdm

d:\machineLearning\miniconda\envs\pytorch-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('-name', type=str, default="nih_25")
parser.add_argument('--dataset_path', type=str, default="imgdata/images-NIH-224/")
parser.add_argument('--num_epochs', type=int, default=1, help='')
parser.add_argument('--batch_size', type=int, default=32, help='')
parser.add_argument('--lr', type=float, default=0.01, help='')
parser.add_argument('--weight_decay', type=float, default=1e-4, help='')
#parser.add_argument('--weight_decay', type=float, default=1e-4, help='')
cfg = parser.parse_args(args=[]) #add this line to use argparse in Notebook

print(cfg)

Namespace(batch_size=32, dataset_path='imgdata/images-NIH-224/', lr=0.01, name='nih_25', num_epochs=1, weight_decay=0.0001)


In [3]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
# define torchvision transforms
data_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.Resize(224),
            # because resize doesn't always give 224 x 224, this ensures 224 x
            # 224
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'val': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
    }


In [4]:
checkpoint_best = torch.load(f'results/{cfg.name}_checkpoint')
model = checkpoint_best['model']

In [5]:
pred_df, metric_res = metrics_model.make_metrics_multilabel(data_transforms, model, cfg)

100%|██████████| 1403/1403 [03:11<00:00,  7.32it/s]


auc
f1
recall
MMC
precision
accuracy
gmean


In [8]:
metric_res_df = pd.DataFrame.from_dict(metric_res).reset_index()

In [9]:
metric_res_df

,index,auc,f1,recall,MMC,precision,accuracy,gmean
0,Atelectasis,0.807973,0.244771,0.161783,0.241258,0.502564,0.527259,0.398304
1,Cardiomegaly,0.894176,0.271605,0.189003,0.290994,0.482456,0.527259,0.433570
2,Consolidation,0.801088,0.004158,0.002090,0.026401,0.400000,0.527259,0.045712
3,Edema,0.888980,0.070981,0.041162,0.096643,0.257576,0.527259,0.202659
4,Effusion,0.877533,0.479898,0.409289,0.429026,0.579949,0.527259,0.626335
5,Emphysema,0.916838,0.299145,0.206287,0.326145,0.544041,0.527259,0.453276
6,Fibrosis,0.817929,0.016260,0.008287,0.057830,0.428571,0.527259,0.091026
7,Hernia,0.887679,0.000000,0.000000,0.000000,0.000000,0.527259,0.000000
8,Infiltration,0.704461,0.161480,0.097363,0.151250,0.472906,0.527259,0.308398
9,Mass,0.828798,0.279926,0.200176,0.282167,0.465306,0.527259,0.444649


In [7]:
 # get preds and AUCs on test fold
preds, aucs = E.make_pred_multilabel(data_transforms, model, cfg)

100%|██████████| 1403/1403 [03:49<00:00,  6.12it/s]


In [8]:
aucs

,label,auc
0,Atelectasis,0.807973
1,Cardiomegaly,0.894176
2,Consolidation,0.801088
3,Edema,0.888980
4,Effusion,0.877533
5,Emphysema,0.916838
6,Fibrosis,0.817929
7,Hernia,0.887679
8,Infiltration,0.704461
9,Mass,0.828798


In [ ]:
preds

In [ ]:
pred_df, true_df = metrics_model.make_metrics_multilabel(data_transforms, model, cfg)

In [ ]:
pred_df

In [ ]:
true_df

In [ ]:
list(preds)[1:]

In [ ]:
preds[preds.columns.drop('Image Index')].to_numpy()